In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from retinaface import RetinaFace
import gc

# Initialize MTCNN
detector = MTCNN()

def preprocess_face(face_img, target_size=(224, 224)):
    try:
        if face_img is None or face_img.size == 0:
            return None

        # Convert to RGB if needed
        if len(face_img.shape) == 2:
            face_img = cv2.cvtColor(face_img, cv2.COLOR_GRAY2RGB)
        elif face_img.shape[2] == 4:
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGRA2RGB)
        elif face_img.shape[2] == 3:
            face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)

        # Resize while maintaining aspect ratio
        aspect_ratio = face_img.shape[1] / face_img.shape[0]
        if aspect_ratio > 1:
            new_width = target_size[0]
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = target_size[1]
            new_width = int(new_height * aspect_ratio)

        resized = cv2.resize(face_img, (new_width, new_height), interpolation=cv2.INTER_AREA)

        # Create blank canvas
        final_img = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)

        # Center the image
        y_offset = (target_size[0] - new_height) // 2
        x_offset = (target_size[1] - new_width) // 2
        final_img[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized

        return final_img

    except Exception as e:
        print(f"Error preprocessing face: {str(e)}")
        return None

def detect_faces(image):
    detections = detector.detect_faces(image)
    face_boxes = [
        detection['box']
        for detection in detections if detection['confidence'] > 0.9
    ]
    return sorted(face_boxes, key=lambda x: x[0])  # Sort left-to-right by x-coordinate

def detect_faces_test(image):
    detections = detector.detect_faces(image)
    face_boxes = [
        detection['box']
        for detection in detections if detection['confidence'] > 0.9
    ]
    if not face_boxes:
        face_boxes = [
        detection['box']
        for detection in detections if detection['confidence'] > 0.7]
    if not face_boxes:
        face_boxes = [
        detection['box']
        for detection in detections if detection['confidence'] > 0.4]
    if not face_boxes:
        face_boxes = [
        detection['box']
        for detection in detections]
    if not face_boxes:
        print("No faces detected. Forcing a default bounding box.")
        height, width = image.shape[:2]
        box_size = min(height, width) // 3  # Default box size as one-third of the image size
        x = (width - box_size) // 2
        y = (height - box_size) // 2
        face_boxes = [[x, y, box_size, box_size]]
    return sorted(face_boxes, key=lambda x: x[0])

In [ ]:
def extract_and_save_faces(images, labels, output_folder, batch_size=50, target_size=(224, 224)):
    os.makedirs(output_folder, exist_ok=True)

    for batch_start in range(0, len(images), batch_size):
        batch_images = images[batch_start:batch_start + batch_size]
        batch_labels = labels[batch_start:batch_start + batch_size]

        for (filename, image), image_labels in zip(batch_images, batch_labels):
            print(f"Processing {filename}...")

            if image_labels == ["nothing"]:
                continue

            # Convert to RGB for detection
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face_boxes = detect_faces(rgb_image)

            # Skip if face count does not match label count
            # if len(face_boxes) != len(image_labels):
            #     print(f"Skipping {filename} - Face count mismatch ({len(face_boxes)} faces, {len(image_labels)} labels)")
            #     continue

            # Process and save each face
            for i, (box, label) in enumerate(zip(face_boxes, image_labels)):
                x, y, w, h = box

                # Add margin to bounding box
                margin = int(max(w, h) * 0.2)
                x = max(0, x - margin)
                y = max(0, y - margin)
                w = min(w + 2 * margin, image.shape[1] - x)
                h = min(h + 2 * margin, image.shape[0] - y)

                # Extract face region
                face = image[y:y+h, x:x+w]

                # if face.size == 0 or face.shape[0] < 50 or face.shape[1] < 50:
                #     print(f"Warning: Face too small in {filename}")
                #     continue

                # Preprocess face
                processed_face = preprocess_face(face, target_size)
                if processed_face is None:
                    print(f"Warning: Could not process face in {filename}")
                    continue

                # Save face image in label folder
                label_folder = os.path.join(output_folder, label.lower())
                os.makedirs(label_folder, exist_ok=True)

                face_filename = f"{os.path.splitext(filename)[0]}_face_{i}.jpg"
                face_path = os.path.join(label_folder, face_filename)
                cv2.imwrite(face_path, cv2.cvtColor(processed_face, cv2.COLOR_RGB2BGR))


In [3]:
def load_images(image_folder, label_map=None):
    images = []
    image_labels = []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)

        if img is not None:
            images.append((filename, img))
            if label_map:
                image_labels.append(label_map.get(filename, []))  # Default to empty list for test images

    return images, image_labels if label_map else None


## Detect images from Trainingset

In [10]:
# Paths
train_image_folder = "../data/olda_data/cleaned_images"
label_csv_path = "../data/labels/clean_data.csv"
output_folder = "../data/faces/train5_faces"

# Load label data
import pandas as pd
label_data = pd.read_csv(label_csv_path)
label_data['label_name'] = label_data['label_name'].apply(eval)  # Convert string to list
label_map = dict(zip(label_data['image'].astype(str).str.zfill(4) + ".jpg", label_data['label_name']))

# Load training images and labels
train_images, train_labels = load_images(train_image_folder, label_map=label_map)

# Crop and save faces
print("Processing training images...")
extract_and_save_faces(train_images, labels=train_labels, output_folder=output_folder, batch_size=50)
print("Cropped faces have been saved.")


Processing training images...
Processing 0032.jpg...


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


# Process Test Images

In [6]:
def extract_and_save_test_faces(images, output_folder, batch_size=50, target_size=(224, 224)):
    os.makedirs(output_folder, exist_ok=True)

    for batch_start in range(0, len(images), batch_size):
        batch_images = images[batch_start:batch_start + batch_size]

        for filename, image in batch_images:
            print(f"Processing {filename}...")

            # Convert to RGB for detection'
            # rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            face_boxes = detect_faces_test(image)

            # Skip images with no detected faces
            if not face_boxes:
                print(f"No faces detected in {filename}. Skipping...")
                continue

            # Process and save each face
            for i, box in enumerate(face_boxes):
                x, y, w, h = box

                # Add margin to bounding box
                margin = int(max(w, h) * 0.2)
                x = max(0, x - margin)
                y = max(0, y - margin)
                w = min(w + 2 * margin, image.shape[1] - x)
                h = min(h + 2 * margin, image.shape[0] - y)

                # Extract face region
                face = image[y:y+h, x:x+w]

                # if face.size == 0 or face.shape[0] < 50 or face.shape[1] < 50:
                #     print(f"Warning: Face too small in {filename}")
                #     continue

                # Preprocess face
                processed_face = preprocess_face(face, target_size)
                if processed_face is None:
                    print(f"Warning: Could not process face in {filename}")
                    continue

                # Save face image in a common folder for test faces
                face_filename = f"{os.path.splitext(filename)[0]}_face_{i}.jpg"
                face_path = os.path.join(output_folder, face_filename)
                cv2.imwrite(face_path, cv2.cvtColor(processed_face, cv2.COLOR_RGB2BGR))


In [7]:
# Paths
test_image_folder = "../data/images/test_images/cleaned_images"
test_output_folder = "../data/faces/test6_faces"

# Load test images
test_images, _ = load_images(test_image_folder)

# Crop and save test faces
print("Processing test images...")
extract_and_save_test_faces(test_images, output_folder=test_output_folder, batch_size=50)
print("Cropped faces from test set have been saved.")


Processing test images...
Processing 0427.jpg...


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
